### Heroes Of Pymoli Data Analysis

Top Line Stats
* Among 576 customers, we have sold 780 items worth $2.4k based on an offering of 183 products. (refer to 'Purchasing Analysis - Total')
* AUR (average unit retail) hovering at $3.05

Gender Demographics
* We are seeing a distinct gender trend, with male-identifying customers making up 84.03% of the customer base.
* Though the customer base is predominantly male-idenftifying, we are seeing more price-resistance from this demographic. They are spending 9% less than non-disclosed customers and 5.6% less than female-identifying customers. They are also spending less than our current AUR of $3.05. Viewed in the light of these trend, there is an argument to be made for working toward growing the non-male customer base.
* These data indicate that each gender is equally likely to make purchases, as their contribution of purchases aligns closely with their portion of the customer pool. It will be interesting to see if we see more distortion as business grows:
    *Female customers: 14.06% of players, 14.5% of purchases
    *Male customers: 84.03% of players, 83.5% of purchases
    *Other/Non-Disclosed customers: 1.91% of players, 1.92% of purchases
    
Age Demographics
* The customer base is concentrated primarily between ages 20-24.
* Because the 20-24 demographic makes up such a large portion of the customer base, their purchases essentially define the AUR ($3.05). The ages demonstrating greater purchasing power are those <10 years-old (average purchase $3.35), and ages 30-34 (average purchase $3.60).
* Viewed through a % to total lens, the 20-24 age group is demonstrating a greater inclination to purchase. Their % of purchases versus total sales exceed the % of the customer base they make up: 46.8% of sales $ / 44.8% of customer base
* Based on these trends, it seems advisable to concentrate marketing to the 20-24 demographic. Not only are they the greatest contributor to total business and largest demographic, they are trending to be the most inclined to purchase. As such, they have the greatest impact on AUR.

Top Spenders
* When viewing the customers who have spent the most in total, we see that they are not price resistant, as each of these customers purchase above AUR on average.

Best Selling Items
* Our top selling item (both in # of units sold and $ volume) is the 'Oathbreaker, Last Hope of the Breaking Storm.' With 12 units sold and a price point of $4.23, this item is key to keeping AUR and volume high.
* The #5 selling item, 'Pursuit, Cudgel of Necromancy', has a low price point of $1.02. All of the items ranking above it are above AUR. Considering its competitive price point, it is suprising to not see it rank higher. Will look forward to seeing how this trend plays out.

In [75]:
# Dependencies and Setup
import pandas as pd
import numpy as np

# Raw data file
file_to_load = "Resources/purchase_data.csv"

# Read purchasing file and store into pandas data frame
purchase_data = pd.read_csv(file_to_load)
purchase_data.head()

,Purchase ID,SN,Age,Gender,Item ID,Item Name,Price
0,0,Lisim78,20,Male,108,"Extraction, Quickblade Of Trembling Hands",3.53
1,1,Lisovynya38,40,Male,143,Frenzied Scimitar,1.56
2,2,Ithergue48,24,Male,92,Final Critic,4.88
3,3,Chamassasya86,24,Male,100,Blindscythe,3.27
4,4,Iskosia90,23,Male,131,Fury,1.44


## Player Count

In [76]:
#count unique SNs to assess the number of active players
user_count = purchase_data['SN'].nunique()
players_count = pd.DataFrame({'Total Players': user_count}, index=[0])
players_count

,Total Players
0,576


## Purchasing Analysis (Total)

In [77]:
#summarize top line information by assessing the related columns, store in DataFrame
item_offer = purchase_data['Item ID'].nunique()
average_price = purchase_data['Price'].mean()
purchase_count = purchase_data['Purchase ID'].count()
total_purchases = purchase_data['Price'].sum()

#generate DataFrame with top line stats
purchase_summary = pd.DataFrame({'Number of Unique Items': item_offer, 
                                 'Average Purchase Price': average_price,
                                'Total Number of Purchases': purchase_count, 
                                 'Total Revenue': total_purchases}, index=[0], columns=['Number of Unique Items', 
                                                                                        'Average Purchase Price', 
                                                                                        'Total Number of Purchases', 
                                                                                        'Total Revenue'])
#set formatting
purchase_summary = purchase_summary.style.format({'Average Purchase Price': '${:,.2f}', 'Total Revenue': '${:,.2f}'})
purchase_summary

,Number of Unique Items,Average Purchase Price,Total Number of Purchases,Total Revenue
0,183,$3.05,780,"$2,379.77"


## Gender Demographics

In [78]:
#create DataFrame containing user counts by gender
gender_demo = pd.DataFrame(purchase_data.groupby('Gender').SN.nunique())

#rename 'SN' column to 'User Count'
gender_demo = gender_demo.rename(index=str, columns={'SN': 'Player Count'})

#add % to totals in a new columns
gender_demo['Percentage of Players'] = gender_demo['Player Count'] / sum(gender_demo['Player Count'])*100

#format new column with %
gender_demo = gender_demo.style.format({'Percentage of Players': '{:.2f}%'})
gender_demo

,Player Count,Percentage of Players
Gender,,
Female,81,14.06%
Male,484,84.03%
Other / Non-Disclosed,11,1.91%



## Purchasing Analysis (Gender)

In [79]:
#create DataFrame to store purchase stats by gender using the 'Purchase ID' and 'Price' columns
gender_stats = pd.DataFrame(purchase_data.groupby('Gender').agg({'Purchase ID':'count', 'Price': {'Total Purchase Value':'sum', 'Average Purchase Price':'mean'}}))

#collapse the multiple indexes to create a cleanly labeled data frame
gender_stats.columns = gender_stats.columns.get_level_values(1) 

#rename the columns to clarify what they are
gender_stats = gender_stats.rename(index=str, columns={'count': 'Purchase Count'})

#add a column to store the average purchase value
gender_stats['Normalized Totals'] = gender_stats['Total Purchase Value']/gender_stats['Purchase Count']

#arrange columns accordingly
gender_stats = gender_stats[['Purchase Count', 'Average Purchase Price', 'Total Purchase Value', 'Normalized Totals']]

#format the 'Total Purchase Value', 'Average Purchase Price', and 'Normalized Totals' columns to display as currency
gender_stats = gender_stats.style.format({'Total Purchase Value': '${:.2f}', 'Average Purchase Price': '${:.2f}', 'Normalized Totals':'${:.2f}'})
gender_stats

C:\Users\creatural1\AppData\Local\Continuum\anaconda3\lib\site-packages\pandas\core\groupby.py:4291: FutureWarning: using a dict with renaming is deprecated and will be removed in a future version
  return super(DataFrameGroupBy, self).aggregate(arg, *args, **kwargs)


,Purchase Count,Average Purchase Price,Total Purchase Value,Normalized Totals
Gender,,,,
Female,113,$3.20,$361.94,$3.20
Male,652,$3.02,$1967.64,$3.02
Other / Non-Disclosed,15,$3.35,$50.19,$3.35


## Age Demographics

In [80]:
# Establish bins for ages
age_bins = [0, 9.90, 14.90, 19.90, 24.90, 29.90, 34.90, 39.90, 99999]
group_names = ["<10", "10-14", "15-19", "20-24", "25-29", "30-34", "35-39", "40+"]

#categorize the players according to the age bin
purchase_data['Age'] = pd.cut(purchase_data['Age'], age_bins, labels=group_names)

#create a DataFrame to store grouped by top-line stats based on age
age_demo = pd.DataFrame(purchase_data.groupby('Age').SN.nunique())

#rename 'SN' column to 'User Count'
age_demo = age_demo.rename(index=str, columns={'SN': 'Player Count'})

#add % to totals in a new columns
age_demo['Percentage of Players'] = age_demo['Player Count'] / sum(age_demo['Player Count'])*100

#format new column with %
age_demo = age_demo.style.format({'Percentage of Players': '{:.2f}%'})
age_demo

,Player Count,Percentage of Players
Age,,
<10,17,2.95%
10-14,22,3.82%
15-19,107,18.58%
20-24,258,44.79%
25-29,77,13.37%
30-34,52,9.03%
35-39,31,5.38%
40+,12,2.08%


## Purchasing Analysis (Age)

In [81]:
#create DataFrame to store purchase stats by age using the 'Purchase ID' and 'Price' columns
age_stats = pd.DataFrame(purchase_data.groupby('Age').agg({'Purchase ID':'count', 'Price': {'Total Purchase Value':'sum', 'Average Purchase Price':'mean'}}))

#collapse the multiple indexes to create a cleanly labeled data frame
age_stats.columns = age_stats.columns.get_level_values(1) 

#rename the columns to clarify what they are
age_stats = age_stats.rename(index=str, columns={'count': 'Purchase Count'})

#add a column to store the average purchase value
age_stats['Normalized Totals'] = age_stats['Total Purchase Value']/age_stats['Purchase Count']

#arrange columns accordingly
age_stats = age_stats[['Purchase Count', 'Average Purchase Price', 'Total Purchase Value', 'Normalized Totals']]

#format the 'Total Purchase Value', 'Average Purchase Price', and 'Normalized Totals' columns to display as currency
age_stats = age_stats.style.format({'Total Purchase Value': '${:.2f}', 'Average Purchase Price': '${:.2f}', 'Normalized Totals':'${:.2f}'})
age_stats

C:\Users\creatural1\AppData\Local\Continuum\anaconda3\lib\site-packages\pandas\core\groupby.py:4291: FutureWarning: using a dict with renaming is deprecated and will be removed in a future version
  return super(DataFrameGroupBy, self).aggregate(arg, *args, **kwargs)


,Purchase Count,Average Purchase Price,Total Purchase Value,Normalized Totals
Age,,,,
<10,23,$3.35,$77.13,$3.35
10-14,28,$2.96,$82.78,$2.96
15-19,136,$3.04,$412.89,$3.04
20-24,365,$3.05,$1114.06,$3.05
25-29,101,$2.90,$293.00,$2.90
30-34,73,$2.93,$214.00,$2.93
35-39,41,$3.60,$147.67,$3.60
40+,13,$2.94,$38.24,$2.94


## Top Spenders

In [82]:
#identify top 5 spenders by grouping purchases by 'SN'
spend_summ = pd.DataFrame(purchase_data.groupby('SN').agg({'Purchase ID':'count', 'Price': {'Total Purchase Value':'sum', 'Average Purchase Price':'mean'}}))

#collapse the multiple indexes to create a cleanly labeled data frame
spend_summ.columns = spend_summ.columns.get_level_values(1) 

#rename the columns to clarify what they are
spend_summ = spend_summ.rename(index=str, columns={'count': 'Purchase Count'})

#sort the 'Total Purchase Value' column in descending order to identify top spenders
spend_summ = spend_summ.sort_values(by='Total Purchase Value', ascending=False)

#slice the DataFrame so that only the top 5 rows populate
spend_summ = spend_summ[0:5]

#convert formatting to currency
spend_summ = spend_summ.style.format({'Total Purchase Value': '${:.2f}', 'Average Purchase Price': '${:.2f}'})
spend_summ

C:\Users\creatural1\AppData\Local\Continuum\anaconda3\lib\site-packages\pandas\core\groupby.py:4291: FutureWarning: using a dict with renaming is deprecated and will be removed in a future version
  return super(DataFrameGroupBy, self).aggregate(arg, *args, **kwargs)


,Purchase Count,Total Purchase Value,Average Purchase Price
SN,,,
Lisosia93,5,$18.96,$3.79
Idastidru52,4,$15.45,$3.86
Chamjask73,3,$13.83,$4.61
Iral74,4,$13.62,$3.40
Iskadarya95,3,$13.10,$4.37


## Most Popular Items

In [86]:
#track most popular items by number of purchases by grouping purchase data according to item id and name
item_data = purchase_data[['Item ID', 'Item Name', 'Price']]

#group the data by item number and name to summarize purchases accordingly
top_purchases = pd.DataFrame(item_data.groupby(['Item ID', 'Item Name']).agg({'Price':['count', 'sum']}))

#colapse the column headers
top_purchases.columns = top_purchases.columns.get_level_values(1)

#add a new column that lists the original price
top_purchases['Item Price'] = top_purchases['sum']/top_purchases['count']

#sort columns
top_purchases = top_purchases[['count', 'Item Price', 'sum']]

#rename columns to reflect the data they are conveying
top_purchases = top_purchases.rename(index=str, columns={'count': 'Purchase Count', 'sum': 'Total Purchase Value'})

#sort the DataFrame according to 'Purchase Count' to identify most popular items
top_purchases = top_purchases.sort_values(by='Purchase Count', ascending=False)

#display top 5 items
top_5_purchases = top_purchases[0:5]

#convert formatting to currency
top_5_purchases = top_5_purchases.style.format({'Total Purchase Value': '${:.2f}', 'Item Price': '${:.2f}'})

top_5_purchases

,,Purchase Count,Item Price,Total Purchase Value
Item ID,Item Name,,,
178,"Oathbreaker, Last Hope of the Breaking Storm",12,$4.23,$50.76
145,Fiery Glass Crusader,9,$4.58,$41.22
108,"Extraction, Quickblade Of Trembling Hands",9,$3.53,$31.77
82,Nirvana,9,$4.90,$44.10
19,"Pursuit, Cudgel of Necromancy",8,$1.02,$8.16


## Most Profitable Items

In [87]:
#use the top purchases data in the previous exercise to identify top volume items
top_purchases = top_purchases.sort_values(by='Total Purchase Value', ascending=False)

#display top 5 items
top_5_volume = top_purchases[0:5]

#convert formatting to currency
top_5_volume = top_5_volume.style.format({'Total Purchase Value': '${:.2f}', 'Item Price': '${:.2f}'})

top_5_volume

,,Purchase Count,Item Price,Total Purchase Value
Item ID,Item Name,,,
178,"Oathbreaker, Last Hope of the Breaking Storm",12,$4.23,$50.76
82,Nirvana,9,$4.90,$44.10
145,Fiery Glass Crusader,9,$4.58,$41.22
92,Final Critic,8,$4.88,$39.04
103,Singed Scalpel,8,$4.35,$34.80
